In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float


In [3]:
"""
Word Processing
"""
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)



In [4]:

"""
TextRNN Parameter
"""
batch_size = len(sentences)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈

def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)



/var/folders/ng/3wjwv6195cb09tvny8_v_7_m0000gn/T/ipykernel_45791/626710555.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1647804319176/work/torch/csrc/utils/tensor_new.cpp:210.)
  input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)


In [5]:

"""
TextLSTM
"""
class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.fc = nn.Linear(n_hidden, n_class)

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    outputs = h_n[-1]  # 최종 예측 Hidden Layer

    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    return model
	


In [6]:
prunFreq = 1

"""
Training
"""
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)


  ### Prune
  if epoch % prunFreq == 0:   
    
    parameters_to_prune = (
    (model.lstm, 'weight_ih_l0'),
    (model.lstm, 'weight_hh_l0'),
    (model.lstm, 'bias_ih_l0'),
    (model.lstm, 'bias_hh_l0'),
    (model.fc, 'weight'),
    (model.fc, 'bias')
    )

    prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.6
    )

    prune.remove(model.lstm, 'weight_ih_l0')
    prune.remove(model.lstm, 'weight_hh_l0')
    prune.remove(model.lstm, 'bias_ih_l0')
    prune.remove(model.lstm, 'bias_hh_l0')
    prune.remove(model.fc, 'weight')
    prune.remove(model.fc, 'bias')


  output = model((hidden, cell), input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model((hidden, cell), input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 0100 cost = 1.112256
Epoch: 0200 cost = 0.648626
Epoch: 0300 cost = 0.326120
Epoch: 0400 cost = 0.150458
Epoch: 0500 cost = 0.085096
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']


In [7]:
list(model.named_parameters())


[('lstm.weight_ih_l0',
  Parameter containing:
  tensor([[-0.0000e+00,  1.9354e+00,  0.0000e+00,  5.9842e-01,  6.7442e-04,
            1.9268e+00, -0.0000e+00,  0.0000e+00,  1.4536e+00],
          [-0.0000e+00,  1.6499e-03,  0.0000e+00, -1.3264e+00, -3.8304e-04,
            1.5551e+00, -0.0000e+00,  0.0000e+00,  1.8822e+00],
          [ 0.0000e+00,  1.7958e+00, -4.1973e-01, -2.1985e+00,  1.6534e-03,
            1.6323e-03,  0.0000e+00, -0.0000e+00, -2.2710e-03],
          [-0.0000e+00,  0.0000e+00, -0.0000e+00,  4.2515e+00, -1.5557e+00,
            2.8564e+00,  0.0000e+00,  3.7646e-01, -7.1255e-03],
          [-0.0000e+00,  7.6235e-03, -3.3528e-01,  2.2035e-04,  1.8952e-03,
            8.6211e-03,  3.6429e-01,  3.7109e-01,  5.5625e-05],
          [-2.9236e-01, -0.0000e+00, -0.0000e+00, -3.1614e+00,  4.0064e+00,
            0.0000e+00,  3.3523e-01, -3.0008e-01, -2.1523e+00],
          [ 0.0000e+00,  2.8739e-01, -3.8624e-01, -1.0625e+00, -3.1511e-04,
           -0.0000e+00,  4.1508e-01, 

In [8]:
model.lstm.weight_ih_l0

Parameter containing:
tensor([[-0.0000e+00,  1.9354e+00,  0.0000e+00,  5.9842e-01,  6.7442e-04,
          1.9268e+00, -0.0000e+00,  0.0000e+00,  1.4536e+00],
        [-0.0000e+00,  1.6499e-03,  0.0000e+00, -1.3264e+00, -3.8304e-04,
          1.5551e+00, -0.0000e+00,  0.0000e+00,  1.8822e+00],
        [ 0.0000e+00,  1.7958e+00, -4.1973e-01, -2.1985e+00,  1.6534e-03,
          1.6323e-03,  0.0000e+00, -0.0000e+00, -2.2710e-03],
        [-0.0000e+00,  0.0000e+00, -0.0000e+00,  4.2515e+00, -1.5557e+00,
          2.8564e+00,  0.0000e+00,  3.7646e-01, -7.1255e-03],
        [-0.0000e+00,  7.6235e-03, -3.3528e-01,  2.2035e-04,  1.8952e-03,
          8.6211e-03,  3.6429e-01,  3.7109e-01,  5.5625e-05],
        [-2.9236e-01, -0.0000e+00, -0.0000e+00, -3.1614e+00,  4.0064e+00,
          0.0000e+00,  3.3523e-01, -3.0008e-01, -2.1523e+00],
        [ 0.0000e+00,  2.8739e-01, -3.8624e-01, -1.0625e+00, -3.1511e-04,
         -0.0000e+00,  4.1508e-01,  4.3023e-01,  4.2076e+00],
        [-3.7172e-01, -0.0

In [9]:
print(
    "Sparsity in lstm.weight_ih_l0: {:.2f}%".format(
        100. * float(torch.sum(model.lstm.weight_ih_l0 == 0))
        / float(model.lstm.weight_ih_l0.nelement())
    )
)

print(
    "Sparsity in lstm.weight_hh_l0: {:.2f}%".format(
        100. * float(torch.sum(model.lstm.weight_ih_l0 == 0))
        / float(model.lstm.weight_ih_l0.nelement())
    )
)

print(
    "Sparsity in lstm.bias_ih_l0: {:.2f}%".format(
        100. * float(torch.sum(model.lstm.bias_ih_l0 == 0))
        / float(model.lstm.bias_ih_l0.nelement())
    )
)

print(
    "Sparsity in lstm.bias_hh_l0: {:.2f}%".format(
        100. * float(torch.sum(model.lstm.bias_hh_l0 == 0))
        / float(model.lstm.bias_hh_l0.nelement())
    )
)

print(
    "Sparsity in fc.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc.weight == 0))
        / float(model.fc.weight.nelement())
    )
)
print(
    "Sparsity in fc.bias: {:.2f}%".format(
        100. * float(torch.sum(model.fc.bias == 0))
        / float(model.fc.bias.nelement())
    )
)


print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.lstm.weight_ih_l0 == 0)
            + torch.sum(model.lstm.weight_hh_l0 == 0)
            + torch.sum(model.lstm.bias_hh_l0 == 0)
            + torch.sum(model.lstm.bias_ih_l0 == 0)
            + torch.sum(model.fc.weight == 0)
            + torch.sum(model.fc.bias == 0)
        )
        / float(
            model.lstm.weight_ih_l0.nelement()
            + model.lstm.weight_hh_l0.nelement()
            + model.lstm.bias_hh_l0.nelement()
            + model.lstm.bias_ih_l0.nelement()
            + model.fc.weight.nelement()
            + model.fc.bias.nelement()
        )
    )
)

Sparsity in lstm.weight_ih_l0: 30.00%
Sparsity in lstm.weight_hh_l0: 30.00%
Sparsity in lstm.bias_ih_l0: 0.00%
Sparsity in lstm.bias_hh_l0: 0.00%
Sparsity in fc.weight: 0.00%
Sparsity in fc.bias: 0.00%
Global sparsity: 14.44%
